# Sequência de Levenshtein

"A distância de Levenshtein é uma métrica do tipo string para medir a diferença entre duas seqüências. Informalmente, a distância de Levenshtein entre duas palavras é o número mínimo de edições de um caractere (inserções, exclusões ou substituições) necessárias para alterar uma palavra pela outra". - Wikipedia

A Distância de Levenshtein possui muitos casos de uso em PNL. No algoritmo mais simples de previsão de palavras, a Distância de Levenshtein é usada para encontrar a palavra mais semelhante. Mesmo na metodologia de correção/detecção ortográfica, o Levenshtein Distance pode ser usado para prever a ortografia correta da palavra a partir de uma determinada dicção. Normalmente, o programa calcula a distância de Levenshtein da palavra-alvo em relação a cada palavra do dicionário. A palavra com o mínimo de distância é priorizada e prevista.

## Uma string possui uma única solução??
Nem sempre, na maioria dos casos, existem múltiplas sequências de Levenshtein entre duas srings com a mesma distância. 

## Escopo e Implementação
Este notebook é focado em encontrar todas as sequências de Levenshtein.

In [1]:
import numpy as np

In [3]:
def getMemorizationMatrix(StringA,StringB):
    matrix = np.zeros([len(StringA)+1,len(StringB)+1])
    
    for j in range(0,len(StringB)+1):
        matrix[0][j] = j
    
    for i in range(0,len(StringA)+1):
        matrix[i][0] = i
        
    for i in range(1,len(StringA)+1):
        for  j in range(1,len(StringB) + 1):
            if(StringA[i-1]==StringB[j-1]):
                matrix[i][j] = matrix[i-1][j-1]
            else:
                matrix[i][j] = np.min([matrix[i-1][j],matrix[i-1][j-1],matrix[i][j-1]])+1
    
    return matrix

def backtrackSequence(matrix,  StringA, StringB, i=None, j=None):
    if(i==None):
        i = len(StringA)
    if(j==None):
        j = len(StringB)
    if(i==0 and j==0):
        return [[[],StringA,0]]
    if(StringA[i-1] == StringB[j-1]):
        paths = backtrackSequence(matrix,StringA,StringB,i-1,j-1)
        return paths
    else:
        paths = []
        if(matrix[i-1][j] + 1 == matrix[i][j] and i-1 >= 0):
            allPaths = backtrackSequence(matrix,StringA,StringB, i-1,j)
            for path in allPaths:
                cS = path[1]
                deviation = path[2]
                path[1] = cS[0:i+deviation-1] + cS[i+deviation:len(cS)]
                path[2] = deviation - 1
                #print(cS + " delete " + StringA[i-1] + " " + path[1] + " " + "("+str(i)+","+str(j)+")")
                path[0].append(cS + " delete " + StringA[i-1] + "("+str(i-1+deviation)+")")
            paths.extend(allPaths)
            #print('Paths',paths)
        if(matrix[i][j-1] + 1 == matrix[i][j] and j-1 >= 0):
            allPaths = backtrackSequence(matrix,StringA,StringB, i,j-1)
            for path in allPaths:
                cS = path[1]
                deviation = path[2]
                path[1] = cS[0:i+deviation] + StringB[j-1] + cS[i+deviation:len(cS)]
                path[2] = deviation + 1
                path[0].append(cS + " insert " + StringB[j-1] +  "("+str(i-1+deviation)+")")
            paths.extend(allPaths)
            #print('Paths',paths)
        if(matrix[i-1][j-1] + 1 == matrix[i][j] and i-1 >= 0 and j-1 >= 0):
            allPaths = backtrackSequence(matrix,StringA,StringB,i-1,j-1)
            for path in allPaths:
                cS = path[1]
                deviation = path[2]
                path[1] = cS[0:i+deviation-1] + StringB[j-1] + cS[i+deviation:len(cS)]
                path[0].append(cS + " replace " + StringA[i-1]  + " with " + StringB[j-1] +  "("+str(i-1+deviation)+")")
            paths.extend(allPaths)
            #print('Paths',paths)
        return paths

def listAllSequence(StringA,StringB):
    matrix = getMemorizationMatrix(StringA,StringB)
    allSequence = backtrackSequence(matrix,StringA,StringB)
    for path in allSequence:
        if(len(path[0])>0):
            print(" -> ".join(path[0])," -> ",path[1])
        else:
            print(StringA," -> ",StringB)

In [4]:
StringA = "paris"
StringB = "alice"

In [7]:
listAllSequence(StringA,StringB)

paris delete p(0) -> aris replace r with l(1) -> alis replace s with c(3) -> alic insert e(3)  ->  alice
paris delete p(0) -> aris replace r with l(1) -> alis insert c(2) -> alics replace s with e(4)  ->  alice


##### Bom Trabalho!